In [2]:
import requests
from bs4 import BeautifulSoup as bs
from  tqdm.notebook import tqdm
from time import sleep
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [19]:
import time

In [248]:
main_link='https://www.captainbook.gr/bookstore/year/2020?page='

headers = {'User-agent': ''}

response = requests.get(main_link, headers=headers)

if response.status_code == 403:
    print('you got cought!')

In [249]:
soup = bs(response.content, 'html.parser')

In [216]:
for_p = soup.find_all('ul', 
                      class_='pagination justify-content-end')[0].find('li', 
                                                                        class_="page_item d-flex align-items-center")

max_pages = for_p.find_all('a')[-2].get_text()

In [217]:
books = []
publisher = []
links = []

In [247]:
price_old = []
price_new = []
price_discount = []

In [254]:
import random

In [255]:
for page in tqdm(range(541,int(max_pages)+1)):
    response = requests.get(main_link+str(page), headers = headers)
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')
        #titles = soup.find_all('h1')
        prices = soup.find_all('div', class_='price p-0 d-flex mb-2 align-items-center justify-content-between')
        #publ = soup.find_all('h2', class_='p-0')
        for i in range(12):
            try: 
                price_old.append(prices[i].find_all('span', class_="old-price p-0")[0].get_text().strip())
            except:
                price_old.append(np.nan)
            try:
                price_new.append(prices[i].find_all('span', class_="new-price p-0")[0].get_text().strip())
            except:
                price_new.append(np.nan)
            try:
                price_discount.append(prices[i].find_all('small', class_="p-0")[0].get_text().strip())
            except:
                price_discount.append(np.nan)
            try:
                publisher.append(publ[i].get_text().strip())
            except:
                publisher.append(np.nan)
            try:
                books.append(titles[i].get_text().strip())
            except:
                books.append(np.nan)
            try:
                links.append(titles[i].find('a', class_="p-0 js-title productClickTracker").get('href'))
            except:
                links.append(np.nan)
    else:
        saved = i
        print('You got caught!')
        break
    
    time.sleep(random.random())   

  0%|          | 0/446 [00:00<?, ?it/s]

In [262]:
dic = {'Title': books,
    'Price_Old': price_old,
    'Discount': price_discount,
    'Price_New': price_new,
    'Publisher': publisher,
    'Link': links }

In [263]:
df = pd.DataFrame(dic, columns=['Title','Price_Old','Discount','Price_New','Publisher','Link'])

In [271]:
df['Year'] = 2020

In [277]:
df

,Title,Price_Old,Discount,Price_New,Publisher,Link,Year
0,Που βρισκόμαστε;,11.00€,-10%,9.90€,Giorgio Agamben,https://www.captainbook.gr/book/pou-briskomast...,2020
1,Η δεκαετία 2011 2020: Ήταν να μην…,17.00€,-10%,15.30€,Συλλογικό έργο,https://www.captainbook.gr/book/tan-na-min-gin...,2020
2,Η ΚΟΛΑΣΗ ΤΗΣ ΤΡΕΜΠΛΙΝΚΑ,13.00€,-10%,11.70€,Vasily Semyonovich Grossman,https://www.captainbook.gr/book/i-kolasi-tis-t...,2020
3,ΟΙ ΕΚΑΤΟ ΜΕΡΕΣ,15.92€,-10%,14.33€,Joseph Roth,https://www.captainbook.gr/book/oi-ekato-meres...,2020
4,Περί θανάτου και άλλων μυστηρίων,7.00€,-10%,6.30€,Fernando Pessoa,https://www.captainbook.gr/book/peri-thanatou-...,2020
...,...,...,...,...,...,...,...
11827,NaN,NaN,NaN,NaN,NaN,NaN,2020
11828,NaN,NaN,NaN,NaN,NaN,NaN,2020
11829,NaN,NaN,NaN,NaN,NaN,NaN,2020
11830,NaN,NaN,NaN,NaN,NaN,NaN,2020


In [278]:
df.shape

(11832, 7)

In [275]:
df.to_csv('books_2020.csv',index=False)

In [280]:
category = []
ISBN = []

for link in tqdm(links):   
    response = requests.get(link, headers = headers)
    if response.status_code == 200:
        soup1 = bs(response.content, 'html.parser')
        body1=soup1.find_all('li', class_="py-2 d-flex justify-content-between")
        try:
            count=0
            pound=0
            
            for body in body1:
                if re.search(r'Κατηγορίες',str(body)):
                    category.append(body.get_text().strip().split('\n')[-1])
                    count+=1
                elif re.search(r'ISBN',str(body)):
                    ISBN.append(body.get_text().strip().split('\n')[-1])
                    pound+=1
            
            if count == 0:
                category.append(np.nan)
            if pound == 0:
                ISBN.append(np.nan)
                
        except:
            pass
        
    else:
        saved=link
        print('You got caught!')
        break
    
    time.sleep(random.uniform(0,4))

  0%|          | 0/11832 [00:00<?, ?it/s]

KeyboardInterrupt: 